In [1]:
import os
import torchaudio
import pandas as pd
from datasets import Dataset
from transformers import Wav2Vec2ForPreTraining, Wav2Vec2Processor, TrainingArguments, Trainer
from tqdm import tqdm
import numpy as np

/home/akhmed.sakip/.conda/envs/nlp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_music_files(directory):
    file_list = [f for f in os.listdir(directory) if f.endswith('.wav')]
    data = []
    for file in tqdm(file_list):
        file_path = os.path.join(directory, file)
        waveform, _ = torchaudio.load(file_path)
        flattened_waveform = waveform.numpy().flatten()
        data.append({"waveform": flattened_waveform, "file_path": file_path})
    return pd.DataFrame(data)


In [3]:
data_directory = "music-segments/"
df = load_music_files(data_directory)
dataset = Dataset.from_pandas(df)

 57%|█████▋    | 9834/17394 [00:28<04:38, 27.13it/s] 

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2ForPreTraining.from_pretrained("facebook/wav2vec2-base")

/home/akhmed.sakip/.conda/envs/nlp/lib/python3.8/site-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
def prepare_dataset(batch):
    waveform = np.expand_dims(batch["waveform"], axis=0)
    batch["input_values"] = processor(waveform, return_tensors="pt", padding=True, max_length=16000, sampling_rate=batch["sampling_rate"]).input_values
    return batch


In [9]:
train_dataset = dataset.map(prepare_dataset, remove_columns=["waveform"], num_proc=1)

Map:   0%|          | 2/17394 [00:00<43:03,  6.73 examples/s]  It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
Map:   0%|          | 4/17394 [00:00<31:37,  9.16 examples/s]It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
Map:   0%|          | 6/17394 [00:00<28:03, 10.33 examples/s]It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommende

KeyboardInterrupt: 